# Excercise 2 - Stretch Objectives

## Objectives
* Put the API request code into a function that can be called repeatedly
* Iterate to make multiple API requests
* Add some basic error handling
* Output the results to the sandpit

In [ ]:
#Imports
import json
import pandas as pd
import requests
from datetime import datetime, timedelta

#New imports
import ncl_sqlsnippets as snips
import time
from sqlalchemy import create_engine

### Make the API Request Function

In [ ]:
#Write a function version of the API request 
##This function can be used to repeatedly make requests from the API
##The function takes a hash_id as an input to specify which site to request data for
##You may want to also convert the response into a tabular form in the function
def request_smart(hash_id):  
    ####ENTER CODE HERE###############
    
    ##################################

### Test your function with the code below

Does the function still work when you change which hash_id you give it?

In [ ]:
df_response = request_smart(hash_id="S0189179")
df_response.head()

### Iterate to make multiple requests

Practically we would want our code to fetch data for all of our sites
We can use a simple loop to do this across multiple requests

This code may take a minute to handle all the requests

In [ ]:
#Full list of sites
site_list = ["S0189179","S89CFECF","S818235B","SF06F50A","S48446E1","S12C3F7C"]

#Define an empty data frame to store the combined results in
df_combined = pd.DataFrame()


for site in site_list:
    new_response = request_smart(hash_id=site)
    time.sleep(5)
    print(f"Processed site {site}")
    df_combined = pd.concat([df_combined, new_response])

In [ ]:
#Check how many rows are in the combined table
df_combined.shape[0]

### Error Handling

Try running the following code

In [ ]:
idx = 0
while(True):
    idx += 1
    print(idx)
    request_smart(hash_id="S0189179")

Did you get an error after a number of requests?

Do you understand what caused this error?

A common secruity measure for APIs is to block sources making an "excessive" number of requests. This is because a common method for cyber attacks is a DDOS attack which works by making endless requests to the website in quick sucession in an attempt to overwhelm the web server and cause the site to crash.

Unfortunately, the SMART API has a fairly low threshold of what it considers to be "excessive" and will block all requests for 1 minute when breached.

To get around this we can add an intentional delay to our code to space out our requests. In the for loop before this was done with the line `time.sleep(5)` which adds a 5 second delay before making the next request.

Another thing we may want to consider is a backup mechanism if we accidentally cross this threshold. Since we know the timeout is 1 minute we can ask the code to wait for a minute when we run into this issue. You can see this implemented below:

In [ ]:
#Full list of sites
site_list = ["S0189179","S89CFECF","S818235B","SF06F50A","S48446E1","S12C3F7C"]

#Define an empty data frame to store the combined results in
df_combined = pd.DataFrame()

#Specify the cool off period
cooloff = 60

for site in site_list:
    #First attempt at making the request
    try:
        new_response = request_smart(hash_id=site)
        print(f"Processed site {site}")
        df_combined = pd.concat([df_combined, new_response])

    except:
        print(f"API Overloaded, waiting before reattempting...")
        time.sleep(cooloff)

        #Second attempt at making the request
        try:
            new_response = request_smart(hash_id=site)
            print(f"Processed site {site}")
            df_combined = pd.concat([df_combined, new_response])
        except:
            raise Exception(
                "Failed twice for the same request so cancelling execution.")

### Output the result to the Sandpit

In [ ]:
#Set up destination parameters

##MAKE SURE YOU DELETE WHAT EVER TEST TABLE YOU CREATE AFTER THE SESSION##

####ENTER CODE HERE###############
sql_address = 'PSFADHSSTP01.AD.ELC.NHS.UK,1460'
sql_database = 'Data_Lab_NCL_Dev'
sql_schema = 'JakeK'
sql_table = 'ds_cop_smart_test'
##################################

#Connect to the sandpit

sql_driver = "SQL+Server"
sql_servertype = "mssql"

connection_string = (f"{sql_servertype}://{sql_address}/{sql_database}"
                     + f"?trusted_connection=yes&driver={sql_driver}")

print("This is the connection string:", connection_string)

engine = create_engine(connection_string, use_setinputsizes=False)

df_combined.to_sql(sql_table, engine, 
                   schema=sql_schema, if_exists="replace", 
                   index=False, chunksize=75, method="multi")

#This can also be done with the ncl_sqlsnippets package with the following code:

#engine = snips.connect(sql_address, sql_database)
#snips.upload_to_sql(df_combined, engine, sql_table, sql_schema, 
#                    replace=True, chunks=75)
